## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-06-20-53-41 +0000,nypost,Supreme Court rules Trump admin can mandate pa...,https://nypost.com/2025/11/06/us-news/supreme-...
1,2025-11-06-20-51-22 +0000,nypost,Mom of four fatally shot after arriving at the...,https://nypost.com/2025/11/06/us-news/cleaning...
2,2025-11-06-20-51-00 +0000,nyt,What Travelers Need to Know About the Shutdown...,https://www.nytimes.com/2025/11/06/travel/shut...
3,2025-11-06-20-50-46 +0000,nyt,Why Is Trump Threatening to Intervene In Nigeria?,https://www.nytimes.com/video/world/africa/100...
4,2025-11-06-20-50-45 +0000,cbc,First budget-related confidence vote expected ...,https://www.cbc.ca/news/politics/budget-confid...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2306/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
3,trump,47
194,mamdani,19
944,new,16
163,election,13
1,court,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
329,2025-11-05-23-44-26 +0000,nyt,Trump vs. Mamdani: President Sees a Capable Fo...,https://www.nytimes.com/2025/11/05/us/politics...,101
59,2025-11-06-19-23-00 +0000,wsj,Former House Speaker Nancy Pelosi says she won...,https://www.wsj.com/politics/nancy-pelosi-reti...,100
105,2025-11-06-17-38-00 +0000,wsj,Many GOP lawmakers are saying no—for now—to Pr...,https://www.wsj.com/politics/policy/senate-rep...,94
291,2025-11-06-04-41-55 +0000,nypost,"Eric Trump calls Zohran Mamdani ‘crazy,’ says ...",https://nypost.com/2025/11/05/us-news/eric-tru...,94
347,2025-11-05-22-51-00 +0000,wsj,President Trump’s global tariffs ran headlong ...,https://www.wsj.com/politics/policy/trumps-tou...,94


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
329,101,2025-11-05-23-44-26 +0000,nyt,Trump vs. Mamdani: President Sees a Capable Fo...,https://www.nytimes.com/2025/11/05/us/politics...
59,100,2025-11-06-19-23-00 +0000,wsj,Former House Speaker Nancy Pelosi says she won...,https://www.wsj.com/politics/nancy-pelosi-reti...
126,73,2025-11-06-17-02-00 +0000,wsj,The FAA said it was ordering airline traffic t...,https://www.wsj.com/business/airlines/u-s-to-r...
200,67,2025-11-06-13-42-00 +0000,wsj,Employers said they cut more than a million jo...,https://www.wsj.com/economy/jobs/planned-u-s-j...
44,48,2025-11-06-19-51-00 +0000,wsj,The Bank of Mexico extended its monetary easin...,https://www.wsj.com/articles/bank-of-mexico-cu...
95,47,2025-11-06-17-53-57 +0000,nypost,"Zohran Mamdani ally Linda Sarsour, NYC DSA pol...",https://nypost.com/2025/11/06/us-news/zohran-m...
336,43,2025-11-05-23-28-53 +0000,nyt,Two Republican Incumbents Will Face Off as Red...,https://www.nytimes.com/2025/11/05/us/politics...
347,42,2025-11-05-22-51-00 +0000,wsj,President Trump’s global tariffs ran headlong ...,https://www.wsj.com/politics/policy/trumps-tou...
346,41,2025-11-05-22-53-33 +0000,nypost,‘Tutors’ at Musk startup xAI had to give up ri...,https://nypost.com/2025/11/05/business/tutors-...
281,37,2025-11-06-06-34-16 +0000,nypost,Typhoon Kalmaegi death toll hits 114 in Philip...,https://nypost.com/2025/11/06/world-news/typho...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
